In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import csv
from lib import *
%matplotlib inline

In [2]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,lib.cnmem=0.7,floatX=float32'

## Bundle model begins

In [ ]:
# folder = '../data/new_data'
# bundle_item_map=pickle.load(open(folder + '/bundle_item_map.pkl','rb'))
# bundle_discount_map = pickle.load(open(folder +'/bundle_discount_map.pkl','rb'))

In [3]:
folder = '../data/train_test'
bpr_item = pickle.load(open(folder +'/bpr_item.pkl','rb'))

In [4]:
H_item=bpr_item.H.eval()
B_item=bpr_item.B.eval()
W_item=bpr_item.W.eval()
TG_item=bpr_item.TG.eval()

In [5]:
folder = '../data/train_test'
sgd_users = pickle.load(open(folder +  "/sgd_users.pkl", "rb" ))
sgd_pos_items = pickle.load(open(folder +  "/sgd_pos_items.pkl", "rb" ))
sgd_neg_items = pickle.load(open(folder +  "/sgd_neg_items.pkl", "rb" ))
sgd_pos_bundles = pickle.load(open(folder +  "/sgd_pos_bundles.pkl", "rb" ))
sgd_neg_bundles = pickle.load(open(folder +  "/sgd_neg_bundles.pkl", "rb" ))
sgd_pos_len = pickle.load(open(folder +  "/sgd_pos_len.pkl", "rb" ))
sgd_neg_len = pickle.load(open(folder +  "/sgd_neg_len.pkl", "rb" ))
sgd_pos_diversity = pickle.load(open(folder +  "/sgd_pos_diversity.pkl", "rb" ))
sgd_neg_diversity = pickle.load(open(folder +  "/sgd_neg_diversity.pkl", "rb" ))

In [6]:
test_users = pickle.load(open(folder +  "/test_users.pkl", "rb" ))
test_pos_items = pickle.load(open(folder +  "/test_pos_items.pkl", "rb" ))
test_neg_items = pickle.load(open(folder +  "/test_neg_items.pkl", "rb" ))
test_pos_bundles = pickle.load(open(folder +  "/test_pos_bundles.pkl", "rb" ))
test_neg_bundles = pickle.load(open(folder +  "/test_neg_bundles.pkl", "rb" ))
test_n1 = pickle.load(open(folder +  "/test_n1.pkl", "rb" ))
test_n2 = pickle.load(open(folder +  "/test_n2.pkl", "rb" ))
test_pos_diversity = pickle.load(open(folder +  "/test_pos_diversity.pkl", "rb" ))
test_neg_diversity =pickle.load(open(folder +  "/test_neg_diversity.pkl", "rb" ))

In [7]:
sgd_pos_items = pickle.load(open(folder +  "/sgd_pos_items.pkl", "rb" ))
sgd_neg_items = pickle.load(open(folder +  "/sgd_neg_items.pkl", "rb" ))

In [8]:
test_pos_bundles = pickle.load(open(folder +  "/test_pos_bundles.pkl", "rb" ))
test_neg_bundles = pickle.load(open(folder +  "/test_neg_bundles.pkl", "rb" ))

In [15]:
item_id_lookup =  pickle.load(open("../data/new_data/item_id_lookup.pkl", "rb"))
user_item_map =  pickle.load(open("../data/new_data/user_item_map.pkl", "rb"))
tag_array = pickle.load(open("../data/new_data/tag_array", "rb"))
bundle_discount_map = pickle.load(open("../data/new_data/bundle_discount_map.pkl", "rb"))
bundle_item_map = pickle.load(open("../data/new_data/bundle_item_map.pkl", "rb"))

In [55]:
max_bundle_size = np.max([len(bundle_item_map[key]) for key in bundle_item_map])
num_items = len(item_id_lookup)
num_users = len(user_item_map)
num_tags = len(tag_array.T)

In [56]:
# folder = '../data/train_test'
# pickle.dump(num_items, open(folder +  "/num_items", "wb" ))
# pickle.dump(max_bundle_size, open(folder +  "/max_bundle_size", "wb" ))
# pickle.dump(num_users, open(folder +  "/num_users", "wb" ))
pickle.dump(num_tags, open(folder +  "/num_tags", "wb" ))

In [57]:
num_items = pickle.load(open(folder +  "/num_items", "r" ))
max_bundle_size = pickle.load(open(folder +  "/max_bundle_size", "r" ))
num_users = pickle.load(open(folder +  "/num_users", "r" ))
num_tags = pickle.load(open(folder +  "/num_tags", "r" ))

In [ ]:
print len(sgd_neg_dicount), len(sgd_pos_diversity), len(sgd_pos_dicount)

In [23]:
H_item = np.concatenate((H_item,np.zeros((1,np.shape(H_item)[1]))),axis=0)
H_item=np.array(H_item).astype('float32')
print np.shape(H_item)


B_item = np.append(B_item,0)
B_item=np.array(B_item).astype('float32')
print np.shape(B_item)

A_item = np.concatenate((TG_item,np.zeros((1,np.shape(TG_item)[1]))),axis=0)
A_item=np.array(A_item).astype('float32')
print np.shape(A_item)

(2518, 10)
(2518,)
(2516, 327)


In [ ]:
#bpr_cold = BPR_Cold(10, max_bundle_size, len(user_bundle_map.keys()), num_items, W_item, H_item, B_item)

In [ ]:
# #bpr_cold.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
#          s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
#                s_neg_diversity=sgd_neg_diversity)

In [ ]:
#bpr_cold.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2, test_pos_diversity, test_neg_diversity)

In [ ]:
print '0.68585732689210954'
bundle_discount_array = [bundle_discount_map[i] for i in range(len(bundle_discount_map)) if i ]

In [ ]:
print 0.73830120101137786 + 'discount devrivate'
print 0.72651723027375203 + 'no discount derivaite'

In [38]:
bundle_discount_arr = [bundle_discount_map[i] for i in range(len(bundle_discount_map))]

In [199]:
class BPR_Buncle1(object):

    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, bundle_discount_arr, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_A = 0.01, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._n_bundle = len(bundle_discount_map)
        self.bundle_discount_arr = theano.shared(np.array(bundle_discount_arr))
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._lambda_A = lambda_A
        self._learning_rate = learning_rate
        
        self._n_tags = n_tags
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        di = T.dvector('di')#cb
        dj = T.dvector('dj')#cb
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.TG1 = theano.shared(A_item.astype('float32'), name='TG')
        self.theta1 = bpr_item.theta
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.BDu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdu')#u
        self.K = theano.shared(numpy.random.rand(), name='K')#k
        self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        self.A1 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A1')
        self.A2 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A2')
                                
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * self.bundle_discount_arr[bi]
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() \
                + self.BDu[u] * self.bundle_discount_arr[bj]
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2)) - self._lambda_A * (self.A1 ** 2).sum() - self._lambda_A * (self.A2 ** 2).sum()
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_A1 = T.grad(cost=cost, wrt=self.A1)
        g_cost_A2 = T.grad(cost=cost, wrt=self.A2)
        g_cost_BDu = T.grad(cost=cost, wrt=self.BDu)

        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.A1, self.A1-self._learning_rate * .001 * g_cost_A1), (self.A2, self.A2-self._learning_rate * .001 * g_cost_A2),
                  (self.BDu, self.BDu-self._learning_rate * .001 * g_cost_BDu)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None, bd_pos_bundle = None, bd_neg_bundle = None, batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal() \
            + self.BDu[u] * self.bundle_discount_arr[bi]
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() \
            + self.BDu[u] * self.bundle_discount_arr[bj]
        y_ui = x_ui - self.BDu[u] * self.bundle_discount_arr[bi]
        y_uj = x_uj - self.BDu[u] * self.bundle_discount_arr[bj]
        
        x_uij = x_ui-x_uj
#         self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=x_uij)
        out = [x_uij, x_ui, x_uj, y_ui, y_uj]
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs= out)

    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len,
                    s_pos_diversity, s_neg_diversity, bd_pos_bundle, bd_neg_bundle, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        x_uis, x_ujs, y_uis, y_ujs = [], [], [], []
        while (z+1)*batch_size < n_sgd_samples:
            pref_list, xu1, xu2, yu1, yu2 =self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            x_uis += [x for x in xu1]
            x_ujs += [x for x in xu2]
            y_uis += [x for x in yu1]
            y_ujs += [x for x in yu2]
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            sys.stderr.write("pref%d" % len(pref_list))
            sys.stderr.flush()
            auc = np.sum([1.0 if a > 0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        sys.stderr.write("\n")
        sys.stderr.flush()
        return [x_uis, x_ujs, y_uis, y_ujs]

In [200]:
bpr_bundle = BPR_Buncle1(10, max_bundle_size, num_users, num_items, \
                        num_tags, bundle_discount_arr)

In [201]:
num_tags = 327
print max_bundle_size, num_users, num_items,num_tags

82 29633 2515 327


In [202]:
# bpr_bundle.train(s_users=sgd_users[:10000], s_pos_items=sgd_pos_items[:10000], s_neg_items=sgd_neg_items[:10000], 
#             s_pos_len=sgd_pos_len[:10000], s_neg_len=sgd_neg_len[:10000], s_pos_diversity=sgd_pos_diversity[:10000],
#             s_neg_diversity=sgd_neg_diversity[:10000], \
#                 bd_pos_bundle = sgd_pos_bundles[:10000], bd_neg_bundle = sgd_neg_bundles[:10000])

Processed 9000 ( 90.00% ) in 0.5506 seconds
Total training time 5.64 seconds; 5.636148e-04 per sample


In [205]:
bpr_bundle.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
            s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
            s_neg_diversity=sgd_neg_diversity, \
                bd_pos_bundle = sgd_pos_bundles, bd_neg_bundle = sgd_neg_bundles)

Processed 2101000 ( 99.99% ) in 0.4112 seconds
Total training time 1111.54 seconds; 5.290189e-04 per sample


In [207]:
res= bpr_bundle.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2,\
                       test_pos_diversity, test_neg_diversity,\
                      test_pos_bundles, test_neg_bundles)

Current AUC mean (420000 samples): 0.93265ndspref1000


In [203]:
# res = bpr_bundle.test_bundle(test_users[:10000], test_pos_items[:10000], test_neg_items[:10000], test_n1[:10000], test_n2[:10000],\
#                        test_pos_diversity[:10000], test_neg_diversity[:10000],\
#                       test_pos_bundles[:10000], test_neg_bundles[:10000])

Current AUC mean (9000 samples): 0.44789ndspref1000


In [252]:
cnt, total = 0, 0
ux1, ux2, uy1, uy2 = res[0], res[1], res[2], res[3]
for x, y in zip(ux1, ux2):
    cnt += 1
    total += 1 if x > y else 0
print total/ float(cnt)
print np.shape(res), np.shape(ux1),np.shape(ux2)

391712 420000
(4, 420000) (420000,) (420000,)


In [257]:
user_bundle_score = {}
for u, x, y, b1, b2 in zip(test_users, ux1, ux2, test_pos_bundles, test_neg_bundles):
    if u not in user_bundle_score:
        user_bundle_score[u] = {}
    user_bundle_score[u][b1] = x 
    user_bundle_score[u][b2] = y

In [258]:
print len(user_bundle_score)

10006


In [88]:
print np.shape(res[0])

(4, 1000)


In [109]:
# bpr_bundle.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2,\
#                        test_pos_diversity, test_neg_diversity,\
#                       test_pos_bundles, test_neg_bundles)

In [106]:
fold = '../data/new_data'
bundle_price_map = pickle.load(open(fold + '/bundle_price_map', 'r'))

In [110]:
bundle_price_arr = [bundle_price_map[i] for i in range(len(bundle_price_map)) if i]

In [241]:
print bpr_bundle.D

D


In [ ]:
print max_bundle_size, len(user_bundle_map.keys()), num_items, len(tag_array.T)

## final price + discount model

In [228]:
class BPR_Buncle2(object):

    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, bundle_discount_arr, bundle_price_arr, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_A = 0.01, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._n_bundle = len(bundle_discount_map)
        self.bundle_discount_arr = theano.shared(np.array(bundle_discount_arr))
        self.bundle_price_arr = theano.shared(np.array(bundle_price_arr))
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._lambda_A = lambda_A
        self._learning_rate = learning_rate
        
        self._n_tags = n_tags
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        di = T.dvector('di')#cb
        dj = T.dvector('dj')#cb
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.TG1 = theano.shared(A_item.astype('float32'), name='TG')
        self.theta1 = bpr_item.theta
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.BDu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdu')#u
        self.BPu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bpu')
        self.K = theano.shared(numpy.random.rand(), name='K')#k
        self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        self.A1 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A1')
        self.A2 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A2')
                                
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * self.bundle_discount_arr[bi] - self.BPu[u] * self.bundle_price_arr[bi]
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() \
                + self.BDu[u] * self.bundle_discount_arr[bj] - self.BPu[u] * self.bundle_price_arr[bj]
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2)) - self._lambda_A * (self.A1 ** 2).sum() - self._lambda_A * (self.A2 ** 2).sum()\
                    
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_A1 = T.grad(cost=cost, wrt=self.A1)
        g_cost_A2 = T.grad(cost=cost, wrt=self.A2)
        g_cost_BDu = T.grad(cost=cost, wrt=self.BDu)
        g_cost_BPu = T.grad(cost=cost, wrt=self.BPu)

        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.A1, self.A1-self._learning_rate * .001 * g_cost_A1), (self.A2, self.A2-self._learning_rate * .001 * g_cost_A2),
                  (self.BDu, self.BDu-self._learning_rate * .001 * g_cost_BDu),
                  (self.BPu, self.BPu-self._learning_rate * .001 * g_cost_BPu)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None, bd_pos_bundle = None, bd_neg_bundle = None, batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal() \
            + self.BDu[u] * self.bundle_discount_arr[bi] - self.BPu[u] * self.bundle_price_arr[bi]
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() \
            + self.BDu[u] * self.bundle_discount_arr[bj] - self.BPu[u] * self.bundle_price_arr[bj]
        y_ui = x_ui - self.BDu[u] * self.bundle_discount_arr[bi] + self.BPu[u] * self.bundle_price_arr[bi]
        y_uj = x_uj - self.BDu[u] * self.bundle_discount_arr[bj] + self.BPu[u] * self.bundle_price_arr[bj]
        
        x_uij = x_ui-x_uj
        #self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=x_uij)
        out = [x_uij, x_ui, x_uj, y_ui, y_uj]
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs= out)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len,
                    s_pos_diversity, s_neg_diversity, bd_pos_bundle, bd_neg_bundle, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        x_uis, x_ujs, y_uis, y_ujs = [], [], [], []
        while (z+1)*batch_size < n_sgd_samples:
            pref_list, xu1, xu2, yu1, yu2 =self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            x_uis += [x for x in xu1]
            x_ujs += [x for x in xu2]
            y_uis += [x for x in yu1]
            y_ujs += [x for x in yu2]
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            sys.stderr.write("pref%d" % len(pref_list))
            sys.stderr.flush()
            auc = np.sum([1.0 if a > 0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        sys.stderr.write("\n")
        sys.stderr.flush()
        return [x_uis, x_ujs, y_uis, y_ujs]

In [229]:
bpr_bundle2 = BPR_Buncle2(10, max_bundle_size, num_users, num_items, \
                        num_tags, bundle_discount_arr, bundle_price_arr)

In [230]:
bpr_bundle2.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
            s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
            s_neg_diversity=sgd_neg_diversity, \
                bd_pos_bundle = sgd_pos_bundles, bd_neg_bundle = sgd_neg_bundles)

Processed 2101000 ( 99.99% ) in 0.3776 seconds
Total training time 778.86 seconds; 3.706853e-04 per sample


In [231]:
res2 = bpr_bundle2.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2,\
                       test_pos_diversity, test_neg_diversity,\
                      test_pos_bundles, test_neg_bundles)
#0.91110n

Current AUC mean (420000 samples): 0.90889ndspref1000


## final price + discount price + final price * discount price model

In [232]:
class BPR_Buncle3(object):

    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, bundle_discount_arr, bundle_price_arr, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_A = 0.01, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._n_bundle = len(bundle_discount_map)
        self.bundle_discount_arr = theano.shared(np.array(bundle_discount_arr))
        self.bundle_price_arr = theano.shared(np.array(bundle_price_arr))
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._lambda_A = lambda_A
        self._learning_rate = learning_rate
        
        self._n_tags = n_tags
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        di = T.dvector('di')#cb
        dj = T.dvector('dj')#cb
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.TG1 = theano.shared(A_item.astype('float32'), name='TG')
        self.theta1 = bpr_item.theta
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.BDu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdu')#discount
        self.BPu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bpu')#final price
        self.BDPu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdpu')#discount price
        self.K = theano.shared(numpy.random.rand(), name='K')#k
        self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        self.A1 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A1')
        self.A2 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A2')
                                
        discount_price_i =  self.bundle_price_arr[bi] / (1 - self.bundle_discount_arr[bi]) * self.bundle_discount_arr[bi]
        discount_price_j =  self.bundle_price_arr[bj] / (1 - self.bundle_discount_arr[bj]) * self.bundle_discount_arr[bj]
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * self.bundle_discount_arr[bi] - self.BPu[u] * self.bundle_price_arr[bi] \
                + self.BDPu[u] * discount_price_i
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() \
                + self.BDu[u] * self.bundle_discount_arr[bj] - self.BPu[u] * self.bundle_price_arr[bj] \
                + self.BDPu[u] * discount_price_j
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2)) - self._lambda_A * (self.A1 ** 2).sum() - self._lambda_A * (self.A2 ** 2).sum()
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_A1 = T.grad(cost=cost, wrt=self.A1)
        g_cost_A2 = T.grad(cost=cost, wrt=self.A2)
        g_cost_BDu = T.grad(cost=cost, wrt=self.BDu)
        g_cost_BPu = T.grad(cost=cost, wrt=self.BPu)
        g_cost_BDPu = T.grad(cost=cost, wrt=self.BDPu)
        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.A1, self.A1-self._learning_rate * .001 * g_cost_A1), (self.A2, self.A2-self._learning_rate * .001 * g_cost_A2),
                  (self.BDu, self.BDu-self._learning_rate * .001 * g_cost_BDu),
                  (self.BPu, self.BPu-self._learning_rate * .001 * g_cost_BPu),
                  (self.BDPu, self.BDPu-self._learning_rate * .001 * g_cost_BDPu)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None, bd_pos_bundle = None, bd_neg_bundle = None, batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal() \
            + self.BDu[u] * self.bundle_discount_arr[bi] - self.BPu[u] * self.bundle_price_arr[bi]
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() \
            + self.BDu[u] * self.bundle_discount_arr[bj] - self.BPu[u] * self.bundle_price_arr[bj]
        y_ui = x_ui - self.BDu[u] * self.bundle_discount_arr[bi] + self.BPu[u] * self.bundle_price_arr[bi]
        y_uj = x_uj - self.BDu[u] * self.bundle_discount_arr[bj] + self.BPu[u] * self.bundle_price_arr[bj]
        
        x_uij = x_ui-x_uj
        #self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=x_uij)
        out = [x_uij, x_ui, x_uj, y_ui, y_uj]
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs= out)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len,
                    s_pos_diversity, s_neg_diversity, bd_pos_bundle, bd_neg_bundle, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        x_uis, x_ujs, y_uis, y_ujs = [], [], [], []
        while (z+1)*batch_size < n_sgd_samples:
            pref_list, xu1, xu2, yu1, yu2 =self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            x_uis += [x for x in xu1]
            x_ujs += [x for x in xu2]
            y_uis += [x for x in yu1]
            y_ujs += [x for x in yu2]
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            sys.stderr.write("pref%d" % len(pref_list))
            sys.stderr.flush()
            auc = np.sum([1.0 if a > 0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        sys.stderr.write("\n")
        sys.stderr.flush()
        return [x_uis, x_ujs, y_uis, y_ujs]

In [233]:
bpr_bundle3 = BPR_Buncle3(10, max_bundle_size, num_users, num_items, \
                        num_tags, bundle_discount_arr, bundle_price_arr)

In [234]:
bpr_bundle3.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
            s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
            s_neg_diversity=sgd_neg_diversity, \
                bd_pos_bundle = sgd_pos_bundles, bd_neg_bundle = sgd_neg_bundles)

Processed 2101000 ( 99.99% ) in 0.3650 seconds
Total training time 834.21 seconds; 3.970270e-04 per sample


In [235]:
res3 = bpr_bundle3.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2,\
                       test_pos_diversity, test_neg_diversity,\
                      test_pos_bundles, test_neg_bundles)

Current AUC mean (420000 samples): 0.88165ndspref1000


## multiplier douscount

In [244]:
class BPR_Buncle4(object):
    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, bundle_discount_arr, bundle_price_arr, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_A = 0.01, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._n_bundle = len(bundle_discount_map)
        self.bundle_discount_arr = theano.shared(np.array(bundle_discount_arr))
        self.bundle_price_arr = theano.shared(np.array(bundle_price_arr))
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._lambda_A = lambda_A
        self._learning_rate = learning_rate
        
        self._n_tags = n_tags
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        di = T.dvector('di')#cb
        dj = T.dvector('dj')#cb
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.TG1 = theano.shared(A_item.astype('float32'), name='TG')
        self.theta1 = bpr_item.theta
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.BDu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdu')#discount
        self.K = theano.shared(numpy.random.rand(), name='K')#k
        self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        self.A1 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A1')
        self.A2 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A2')
                                
        discount_price_i =  self.bundle_price_arr[bi] / (1 - self.bundle_discount_arr[bi]) * self.bundle_discount_arr[bi]
        discount_price_j =  self.bundle_price_arr[bj] / (1 - self.bundle_discount_arr[bj]) * self.bundle_discount_arr[bj]
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * T.nnet.sigmoid(theano.tensor.tan(self.bundle_discount_arr[bi] * theano.tensor.constant(np.pi).eval()))
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * T.nnet.sigmoid(theano.tensor.tan(self.bundle_discount_arr[bj] * theano.tensor.constant(np.pi).eval()))
        
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2)) - self._lambda_A * (self.A1 ** 2).sum() - self._lambda_A * (self.A2 ** 2).sum()
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_A1 = T.grad(cost=cost, wrt=self.A1)
        g_cost_A2 = T.grad(cost=cost, wrt=self.A2)
        g_cost_BDu = T.grad(cost=cost, wrt=self.BDu)
        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.A1, self.A1-self._learning_rate * .001 * g_cost_A1), (self.A2, self.A2-self._learning_rate * .001 * g_cost_A2),
                  (self.BDu, self.BDu-self._learning_rate * .001 * g_cost_BDu)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None, bd_pos_bundle = None, bd_neg_bundle = None, batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * T.nnet.sigmoid(theano.tensor.tan(self.bundle_discount_arr[bi] * theano.tensor.constant(np.pi).eval()))
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * T.nnet.sigmoid(theano.tensor.tan(self.bundle_discount_arr[bj] * theano.tensor.constant(np.pi).eval()))
        y_ui = x_ui - self.BDu[u] * T.nnet.sigmoid(theano.tensor.tan(self.bundle_discount_arr[bi] * theano.tensor.constant(np.pi).eval()))
        y_uj = x_uj - self.BDu[u] * T.nnet.sigmoid(theano.tensor.tan(self.bundle_discount_arr[bj] * theano.tensor.constant(np.pi).eval()))
        
        x_uij = x_ui-x_uj
        #self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=x_uij)
        out = [x_uij, x_ui, x_uj, y_ui, y_uj]
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs= out)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len,
                    s_pos_diversity, s_neg_diversity, bd_pos_bundle, bd_neg_bundle, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        x_uis, x_ujs, y_uis, y_ujs = [], [], [], []
        while (z+1)*batch_size < n_sgd_samples:
            pref_list, xu1, xu2, yu1, yu2 =self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            x_uis += [x for x in xu1]
            x_ujs += [x for x in xu2]
            y_uis += [x for x in yu1]
            y_ujs += [x for x in yu2]
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            sys.stderr.write("pref%d" % len(pref_list))
            sys.stderr.flush()
            auc = np.sum([1.0 if a > 0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        sys.stderr.write("\n")
        sys.stderr.flush()
        return [x_uis, x_ujs, y_uis, y_ujs]

In [245]:
bpr_bundle4 = BPR_Buncle4(10, max_bundle_size, num_users, num_items, \
                        num_tags, bundle_discount_arr, bundle_price_arr)

In [246]:
bpr_bundle4.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
            s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
            s_neg_diversity=sgd_neg_diversity, \
                bd_pos_bundle = sgd_pos_bundles, bd_neg_bundle = sgd_neg_bundles)

Processed 2101000 ( 99.99% ) in 0.3186 seconds
Total training time 847.38 seconds; 4.032961e-04 per sample


In [247]:
res4 = bpr_bundle4.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2,\
                       test_pos_diversity, test_neg_diversity,\
                      test_pos_bundles, test_neg_bundles)

Current AUC mean (420000 samples): 0.93170ndspref1000


In [248]:
class BPR_Buncle5(object):
    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, bundle_discount_arr, bundle_price_arr, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_A = 0.01, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._n_bundle = len(bundle_discount_map)
        self.bundle_discount_arr = theano.shared(np.array(bundle_discount_arr))
        self.bundle_price_arr = theano.shared(np.array(bundle_price_arr))
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._lambda_A = lambda_A
        self._learning_rate = learning_rate
        
        self._n_tags = n_tags
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        di = T.dvector('di')#cb
        dj = T.dvector('dj')#cb
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.TG1 = theano.shared(A_item.astype('float32'), name='TG')
        self.theta1 = bpr_item.theta
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.BDu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdu')#discount
        self.BDu2 = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdu2')#discount

        self.K = theano.shared(numpy.random.rand(), name='K')#k
        self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        self.A1 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A1')
        self.A2 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A2')
                                
        discount_price_i =  self.bundle_price_arr[bi] / (1 - self.bundle_discount_arr[bi]) * self.bundle_discount_arr[bi]
        discount_price_j =  self.bundle_price_arr[bj] / (1 - self.bundle_discount_arr[bj]) * self.bundle_discount_arr[bj]
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * self.bundle_discount_arr[bi] + self.BDu2[u] * self.bundle_discount_arr[bi] ** 2
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * self.bundle_discount_arr[bj] + self.BDu2[u] * self.bundle_discount_arr[bj] ** 2
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2)) - self._lambda_A * (self.A1 ** 2).sum() - self._lambda_A * (self.A2 ** 2).sum()
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_A1 = T.grad(cost=cost, wrt=self.A1)
        g_cost_A2 = T.grad(cost=cost, wrt=self.A2)
        g_cost_BDu = T.grad(cost=cost, wrt=self.BDu)
        g_cost_BDu2 = T.grad(cost=cost, wrt=self.BDu2)
        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.A1, self.A1-self._learning_rate * .001 * g_cost_A1), (self.A2, self.A2-self._learning_rate * .001 * g_cost_A2),
                  (self.BDu, self.BDu-self._learning_rate * .001 * g_cost_BDu),
                  (self.BDu2, self.BDu2-self._learning_rate * .001 * g_cost_BDu2)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None, bd_pos_bundle = None, bd_neg_bundle = None, batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        bi = T.lvector('bi')
        bj = T.lvector('bj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * self.bundle_discount_arr[bi] + self.BDu2[u] * self.bundle_discount_arr[bi] ** 2
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal()\
                + self.BDu[u] * self.bundle_discount_arr[bj] + self.BDu2[u] * self.bundle_discount_arr[bj] ** 2
        y_ui = x_ui - self.BDu[u] * self.bundle_discount_arr[bi] + self.BDu2[u] * self.bundle_discount_arr[bi] ** 2
        y_uj = x_uj - self.BDu[u] * self.bundle_discount_arr[bj] + self.BDu2[u] * self.bundle_discount_arr[bj] ** 2
       
        x_uij = x_ui-x_uj
        #self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs=x_uij)
        out = [x_uij, x_ui, x_uj, y_ui, y_uj]
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bi, bj], outputs= out)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len,
                    s_pos_diversity, s_neg_diversity, bd_pos_bundle, bd_neg_bundle, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        x_uis, x_ujs, y_uis, y_ujs = [], [], [], []
        while (z+1)*batch_size < n_sgd_samples:
            pref_list, xu1, xu2, yu1, yu2 =self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_bundle[z*batch_size: (z+1)*batch_size],
                bd_neg_bundle[z*batch_size: (z+1)*batch_size]
            )
            x_uis += [x for x in xu1]
            x_ujs += [x for x in xu2]
            y_uis += [x for x in yu1]
            y_ujs += [x for x in yu2]
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            sys.stderr.write("pref%d" % len(pref_list))
            sys.stderr.flush()
            auc = np.sum([1.0 if a > 0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        sys.stderr.write("\n")
        sys.stderr.flush()
        return [x_uis, x_ujs, y_uis, y_ujs]

In [249]:
bpr_bundle5 = BPR_Buncle5(10, max_bundle_size, num_users, num_items, \
                        num_tags, bundle_discount_arr, bundle_price_arr)

In [250]:
bpr_bundle5.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
            s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
            s_neg_diversity=sgd_neg_diversity, \
                bd_pos_bundle = sgd_pos_bundles, bd_neg_bundle = sgd_neg_bundles)

Processed 2101000 ( 99.99% ) in 0.3248 seconds
Total training time 738.46 seconds; 3.514578e-04 per sample


In [251]:
res5 = bpr_bundle5.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2,\
                       test_pos_diversity, test_neg_diversity,\
                      test_pos_bundles, test_neg_bundles)

Current AUC mean (420000 samples): 0.93158ndspref1000
